# Imports

In [1]:
import gym
from utils import *
from agent_arena_no_gathering import *
from environments.snake.snake_env import Snake
from environments.windy_grid_world.windy_grid_world_env import WindyGridWorld
from environments.evil_wgw_env import EvilWindyGridWorld

/root/miniconda3/envs/opensim-rl/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# Snake Environment

### Environment initializtion

In [2]:
#env = Snake(grid_size=(8, 8))
#num_actions = 3
env = EvilWindyGridWorld(grid_size=(7, 10), visual=True, stochasticity=0.01)
num_actions = 4

### Agent training

In [ ]:
for i in range(5):
    for stoc in [0.0, 0.02, 0.05, 0.1, 0.2]:
        
        tf.reset_default_graph()
    
        dqn_trainer = DQNTrainer(
            num_actions, state_shape=[10, 7, 1],
            convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[64, 64],
            gradient_clip=10.0,
            scope="dqn_trainer")

        quant_trainer = QuantileTrainer(
            num_actions, state_shape=[10, 7, 1],
            convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[64],
            num_atoms=64, kappa=1.0,
            scope="qr_trainer")
        
        cat_trainer = CategoricalTrainer(
            num_actions, state_shape=[10, 7, 1],
            convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[64],
            num_atoms=64, v=(-5, 5),
            scope="cat_trainer")

        trainers = [quant_trainer, dqn_trainer, cat_trainer]
    
        env = EvilWindyGridWorld(
            grid_size=(7, 10), visual=True, stochasticity=stoc)
        test_env = EvilWindyGridWorld(
            grid_size=(7, 10), visual=True, stochasticity=stoc)

        agent_arena = AgentArena(
            env, test_env, num_actions, trainers,
            save_path="evil_wgw_models/dec17/exp"+str(stoc)[2:]+"_"+str(i+1))
    
        agent_arena.set_parameters(
            max_episode_length=250, replay_memory_size=100000,
            replay_start_size=250, discount_factor=0.99,
            final_eps=0.01, annealing_steps=200000)
    
        agent_arena.start(
            gpu_id=7, performance_print_freq=20, max_num_frames=2000000)

frame count: 5000
average reward: 0.0
epsilon: 0.975
-------------------------------


In [3]:
dqn_trainer1 = DQNTrainer(
    num_actions, state_shape=[10, 7, 1],
    convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[64],
    scope="dqn1")

dqn_trainer2 = DQNTrainer(
    num_actions, state_shape=[10, 7, 1],
    convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[64],
    scope="dqn2")

qr_trainer = QRTrainer(
    num_actions, state_shape=[10, 7, 1],
    convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[64],
    num_atoms=64, kappa=1.0,
    scope="qr2")

trainers = [dqn_trainer1, dqn_trainer2, qr_trainer]

In [4]:
env = EvilWindyGridWorld(grid_size=(7, 10), visual=True, stochasticity=0.05)
test_env = EvilWindyGridWorld(grid_size=(7, 10), visual=True, stochasticity=0.05)

agent_arena = AgentArena(env, test_env, num_actions, trainers, save_path="arena")

In [5]:
agent_arena.set_parameters(
    max_episode_length=100, replay_memory_size=100000, replay_start_size=500,
    discount_factor=0.99, final_eps=0.01, annealing_steps=200000)

(59, 10, 7, 1)


In [ ]:
agent_arena.start(gpu_id=6, performance_print_freq=20, max_num_episodes=2000)

In [ ]:
# Create basic agent which consists of two networks: agent and target.
# Checkpoints of networks' weights and learning curves will be saved
# in "save_path/model_name" folder.
wgw_agent = DQNAgent(
    env, num_actions, state_shape=[10, 7, 1],
    convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[64],
    save_path="evil_wgw_models", model_name="dqn_4")

In [ ]:
wgw_agent = CatDQNAgent(
    env, num_actions, state_shape=[10, 7, 1],
    convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[64],
    v=(-5, 5), num_atoms=11,
    save_path="wgw_models", model_name="catdqn_11")

In [ ]:
wgw_agent = QuantRegDQNAgent(
    env, num_actions, state_shape=[10, 7, 1],
    convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[64],
    num_atoms=4,
    optimizer=tf.train.AdamOptimizer(2.5e-4, epsilon=0.01/32),
    save_path="wgw_models", model_name="quantdqn4_stoc03")

In [ ]:
# Set basic hyper parameters (for full list see "set_parameters" method).
# Create replay buffer and fill it with random transitions.
wgw_agent.set_parameters(max_episode_length=250, replay_memory_size=100000, replay_start_size=500,
                           discount_factor=0.99, final_eps=0.01, annealing_steps=200000)

In [ ]:
# Set training hyper parameters (for full list see "train" method).
# Set gpu_id = -1 to use cpu instead if gpu, otherwise set it to gpu device id.
wgw_agent.train(gpu_id=4, exploration="e-greedy", save_freq=1000, max_num_epochs=5000, performance_print_freq=20)

In [ ]:
#num_atoms_range = [2, 4, 8, 16, 32, 64]
num_atoms_range = [64] * 6
for i, num_atoms in enumerate(num_atoms_range):
    env = EvilWindyGridWorld(grid_size=(7, 10), visual=True, stochasticity=0.05)
    num_actions = 4
    
    wgw_agent = QuantRegDQNAgent(
        env, num_actions, state_shape=[10, 7, 1],
        convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[64],
        num_atoms=num_atoms,
        optimizer=tf.train.AdamOptimizer(2.5e-4, epsilon=0.01/32),
        save_path="evil_wgw_models", model_name="quant64_"+str(i+10))
    
    #wgw_agent = DQNAgent(
    #    env, num_actions, state_shape=[10, 7, 1],
    #    convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[64],
    #    save_path="evil_wgw_models", model_name="simple_dqn_"+str(i+1))

    # Set basic hyper parameters (for full list see "set_parameters" method).
    # Create replay buffer and fill it with random transitions.
    wgw_agent.set_parameters(
        max_episode_length=250, replay_memory_size=100000, replay_start_size=250,
        discount_factor=0.99, final_eps=0.01, annealing_steps=100000)

    # Set training hyper parameters (for full list see "train" method).
    # Set gpu_id = -1 to use cpu instead if gpu, otherwise set it to gpu device id.
    wgw_agent.train(
        gpu_id=0, exploration="e-greedy", save_freq=1000,
        max_num_epochs=1500, performance_print_freq=20)

### Other agents

In [ ]:
# Classic deep Q-network
snake_agent = DQNAgent(env, num_actions, state_shape=[8, 8, 5],
                       convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[128],
                       save_path="snake_models", model_name="dqn_8x8")

# Dueling deep Q-network
snake_agent = DuelDQNAgent(env, num_actions, state_shape=[8, 8, 5],
                           convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[64],
                           save_path="snake_models", model_name="dueldqn_8x8")

# Categorical deep Q-network (C51)
snake_agent = CatDQNAgent(env, num_actions, state_shape=[8, 8, 5],
                          convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[128],
                          v=(-5, 25), num_atoms=51,
                          save_path="snake_models", model_name="catdqn_8x8")

# Quantile regression deep Q-network (QR-DQN)
snake_agent = QuantRegDQNAgent(env, num_actions, state_shape=[8, 8, 5],
                               convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[128],
                               num_atoms=100, kappa=1.0,
                               save_path="snake_models", model_name="quantdqn_8x8")

# Soft Actor-Critic
snake_agent = SACAgent(env, num_actions, state_shape=[8, 8, 5],
                       convs=[[16, 2, 1], [32, 1, 1]], fully_connected=[128],
                       temperature=0.1,
                       save_path="snake_models", model_name="sac_8x8")

# Atari Environment

### Environment initializtion

In [ ]:
game_id = "PongNoFrameskip-v4"
env = wrap_deepmind(gym.make(game_id))
num_actions = env.unwrapped.action_space.n

### Agent training

In [ ]:
atari_agent = DQNAgent(env, num_actions, state_shape=[84, 84, 4],
                       convs=[[32, 8, 4], [64, 4, 2], [64, 3, 1]], fully_connected=[512],
                       save_path="atari_models", model_name="dqn_boi")

In [ ]:
atari_agent.set_parameters(max_episode_length=100000, discount_factor=0.99, final_eps=0.01,
                           replay_memory_size=1000000, replay_start_size=50, annealing_steps=1000000,
                           frame_history_len=4)

In [ ]:
atari_agent.train(gpu_id=-1, exploration="e-greedy", save_freq=50000, 
                  max_num_epochs=1000, performance_print_freq=50)

### Other agents

In [ ]:
# Classic deep Q-network
atari_agent = DQNAgent(env, num_actions, state_shape=[84, 84, 4],
                       convs=[[32, 8, 4], [64, 4, 2], [64, 3, 1]], fully_connected=[512],
                       save_path="atari_models", model_name="dqn_boi")

# Dueling deep Q-network
atari_agent = DuelDQNAgent(env, num_actions, state_shape=[84, 84, 4],
                           convs=[[32, 8, 4], [64, 4, 2], [64, 3, 1]], fully_connected=[256],
                           save_path="atari_models", model_name="dueldqn_boi")

# Categorical deep Q-network (C51)
atari_agent = CatDQNAgent(env, num_actions, state_shape=[84, 84, 4],
                          convs=[[32, 8, 4], [64, 4, 2], [64, 3, 1]], fully_connected=[512],
                          v=(-10, 10), num_atoms=51,
                          save_path="atari_models", model_name="catdqn_boi")

# Quantile regression deep Q-network (QR-DQN)
atari_agent = QuantRegDQNAgent(env, num_actions, state_shape=[84, 84, 4],
                               convs=[[32, 8, 4], [64, 4, 2], [64, 3, 1]], fully_connected=[512],
                               num_atoms=200, kappa=1,
                               save_path="atari_models", model_name="quantdqn_boi")